In [1]:
%pip install torch transformers peft accelerate datasets bert-score pandas tqdm tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 364.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 376.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 178.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 308.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 238.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 388.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 383.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 391.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 333.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 444.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 388.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you

In [2]:
# -*- coding: utf-8 -*-
"""
(수정-v2) 베이스 모델과 파인튜닝된 모델의 답변 품질을 정성적으로 비교 평가하기 위한 스크립트.

변경 사항:
- 최종 목표인 '새로운 디자인 생성' 능력에 초점을 맞춤.
- 창의적인 컨셉을 제시하고, 그에 맞는 구체적인 디자인 요소를 '생성'하도록 유도하는 질문으로 구성.
"""

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import pandas as pd
from tqdm import tqdm

# --- 설정 ---
BASE_MODEL_PATH = "./exaone_4.0_1.2b"
ADAPTER_PATH = "./llm_finetuned_model"
OUTPUT_MD_PATH = "qualitative_comparison_v5.md"

# 새로운 디자인을 '생성'하는 능력을 평가하기 위한 질문 목록
QUESTIONS = [
    # --- 컨셉 기반 신규 디자인 생성 ---
    "'한국의 전통적인 한옥과 수묵화'를 컨셉으로 제네시스 G90의 새로운 스페셜 에디션 모델을 디자인해줘. 특히 크레스트 그릴, 휠, 실내 내장재의 디자인이 어떻게 바뀔지 구체적으로 묘사해줘.",
    "2050년 미래 해양 도시를 탐험하기 위한 '현대 포세이돈'이라는 이름의 수륙양용 SUV를 상상해서 디자인해줘. 공기역학적인 차체, 잠수 모드를 위한 헤드라이트, 물 속 추진을 위한 휠의 변형 디자인을 중심으로.",
    
    # --- 특정 디자인 요소의 창의적 융합 및 재해석 ---
    "현대자동차의 '파라메트릭 픽셀'과 제네시스의 '두 줄' 디자인을 융합해서, 새로운 전기 스포츠카의 테일램프 디자인을 만들어줘. 어떤 모양일지 아주 상세하게 설명해줘.",
    
    # --- 브랜드 아이덴티티 기반의 새로운 모델 생성 ---
    "현대의 고성능 'N' 브랜드에서 최초의 오프로드용 픽업트럭을 만든다면 어떤 모습일까? 'N' 브랜드의 상징색, 공격적인 범퍼 디자인, 그리고 거친 지형을 위한 타이어와 휠 디자인을 구체적으로 설명해줘."
]

def load_model(model_path, adapter_path=None):
    """모델과 토크나이저를 로드하는 통합 함수"""
    print(f"모델 로드 중: {model_path}" + (f" + {adapter_path}" if adapter_path else ""))
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        trust_remote_code=True,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    if adapter_path:
        model = PeftModel.from_pretrained(model, adapter_path)
        model = model.merge_and_unload()
        print("✅ 파인튜닝 모델 로드 및 병합 완료")
    else:
        print("✅ 베이스 모델 로드 완료")
        
    return model, tokenizer

def generate_answer(model, tokenizer, question):
    """주어진 모델과 질문으로 답변을 생성하는 함수"""
    messages = [{"role": "user", "content": question}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    try:
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        
        # 창의적이고 상세한 생성을 위해 옵션 유지
        output = model.generate(
            input_ids, 
            max_new_tokens=512, 
            do_sample=True, 
            temperature=0.8,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1 # 반복을 줄여 좀 더 창의적인 결과 유도
        )
        
        full_text = tokenizer.decode(output[0], skip_special_tokens=True)
        answer = full_text.split("assistant\n")[1].strip() if "assistant\n" in full_text else full_text

        answer = answer.replace("\n", "<br>").replace("|", "&#124;")
        return answer
    except Exception as e:
        return f"답변 생성 중 오류 발생: {str(e)}"

def main():
    """메인 실행 함수"""
    base_model, tokenizer = load_model(BASE_MODEL_PATH)
    base_answers = []
    print("\n--- 베이스 모델 답변 생성 시작 ---")
    for q in tqdm(QUESTIONS, desc="베이스 모델"):
        base_answers.append(generate_answer(base_model, tokenizer, q))
    del base_model
    torch.cuda.empty_cache()

    ft_model, tokenizer = load_model(BASE_MODEL_PATH, ADAPTER_PATH)
    ft_answers = []
    print("\n--- 파인튜닝 모델 답변 생성 시작 ---")
    for q in tqdm(QUESTIONS, desc="파인튜닝 모델"):
        ft_answers.append(generate_answer(ft_model, tokenizer, q))
    del ft_model
    torch.cuda.empty_cache()

    print(f"\n--- 결과를 {OUTPUT_MD_PATH} 파일로 저장 중 ---")
    with open(OUTPUT_MD_PATH, 'w', encoding='utf-8') as f:
        f.write("# 모델별 답변 정성 평가 (v2 - 창의적 디자인 생성 중심)\n\n")
        f.write("| 질문 (Creative Brief) | 베이스 모델 답변 (Base Model) | 파인튜닝 모델 답변 (Finetuned Model) |\n")
        f.write("|---|---|---|")
        for i in range(len(QUESTIONS)):
            f.write(f"| {QUESTIONS[i]} | {base_answers[i]} | {ft_answers[i]} |\n")

    print(f"✅ 정성 평가용 데이터 생성이 완료되었습니다. '{OUTPUT_MD_PATH}' 파일을 확인해주세요.")

if __name__ == "__main__":
    main()


모델 로드 중: ./exaone_4.0_1.2b


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 베이스 모델 로드 완료

--- 베이스 모델 답변 생성 시작 ---


베이스 모델: 100%|██████████| 4/4 [00:48<00:00, 12.21s/it]


모델 로드 중: ./exaone_4.0_1.2b + ./llm_finetuned_model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 파인튜닝 모델 로드 및 병합 완료

--- 파인튜닝 모델 답변 생성 시작 ---


파인튜닝 모델: 100%|██████████| 4/4 [00:49<00:00, 12.39s/it]


--- 결과를 qualitative_comparison_v5.md 파일로 저장 중 ---
✅ 정성 평가용 데이터 생성이 완료되었습니다. 'qualitative_comparison_v5.md' 파일을 확인해주세요.


In [6]:
# -*- coding: utf-8 -*-
"""
개선된 프롬프트(Chat Template)를 사용하여 모델의 정량 평가를 수행하는 스크립트.

기존 방식의 한계:
- `Question: {question}Answer:` 형식의 단순한 프롬프트는
  모델이 가진 본래의 대화/지시사항 처리 능력을 제대로 활용하지 못할 수 있습니다.

개선된 방식:
- `tokenizer.apply_chat_template`을 사용하여 모델이 학습된 형식에 맞는
  최적의 프롬프트를 자동으로 생성합니다. 이를 통해 모델의 성능을 더 정확하게 측정할 수 있습니다.
- 각 모델(베이스, 파인튜닝)의 평가를 별도의 함수로 분리하여 코드의 명확성과 재사용성을 높였습니다.
"""

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import bert_score
import pandas as pd
from tqdm import tqdm

# --- 공통 설정 ---
BASE_MODEL_PATH = "./exaone_4.0_1.2b"
ADAPTER_PATH = "./llm_finetuned_model"
TEST_DATA_PATH = './test.jsonl'

def evaluate_model(model_name, model, tokenizer, data):
    """주어진 모델에 대해 평가를 수행하고 결과를 반환하는 함수"""
    print(f"\n--- [{model_name}] 모델 답변 생성 시작 ---")
    
    predictions = []
    references = []
    
    for item in tqdm(data, desc=f"평가 중 [{model_name}]"):
        question = item['messages'][0]['content']
        reference_answer = item['messages'][1]['content']
        
        # 개선된 프롬프트: tokenizer의 chat template 사용
        messages = [{"role": "user", "content": question}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        
        # do_sample=False는 기존 노트북과 동일한 환경을 위함. True로 바꾸면 더 나은 결과가 나올 수 있음.
        output = model.generate(
            input_ids, 
            max_new_tokens=256, 
            do_sample=False, 
            eos_token_id=tokenizer.eos_token_id
        )
        
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        # 프롬프트 부분을 제거하고 답변만 추출 (chat template 결과에 따라 후처리 방식이 달라질 수 있음)
        answer = generated_text[len(prompt):].strip()
        
        predictions.append(answer)
        references.append(reference_answer)

    print(f"--- [{model_name}] BERTScore 계산 중 ---")
    bert_p, bert_r, bert_f1 = bert_score.score(
        predictions, references, lang="ko", model_type="bert-base-multilingual-cased", verbose=False
    )
    
    results = []
    for i in range(len(predictions)):
        results.append({
            "Question": data[i]['messages'][0]['content'],
            "Reference Answer": references[i],
            "Generated Answer": predictions[i],
            "BERTScore-P": bert_p[i].item(),
            "BERTScore-R": bert_r[i].item(),
            "BERTScore-F1": bert_f1[i].item()
        })
        
    print(f"✅ [{model_name}] 평가 완료")
    return pd.DataFrame(results)

def main():
    """메인 실행 함수"""
    # --- 데이터 로드 ---
    with open(TEST_DATA_PATH, 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f if line.strip()]
    print(f"✅ 총 {len(test_data)}개의 평가 데이터를 로드했습니다.")

    # --- 1. 베이스 모델 평가 ---
    base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
    
    base_results_df = evaluate_model("Base Model", base_model, tokenizer, test_data)
    
    # 결과 저장 및 출력
    base_avg_scores = base_results_df[[col for col in base_results_df.columns if "BERTScore" in col]].mean()
    print("\n--- [Base Model] 전체 평균 점수 ---\
", base_avg_scores)
    base_results_df.to_markdown("./base_model_evaluation02.md", index=False)
    print("✅ 베이스 모델 평가 결과가 improved_base_model_evaluation02.md 파일에 저장되었습니다.")
    
    del base_model
    torch.cuda.empty_cache()

    # --- 2. 파인튜닝 모델 평가 ---
    # 베이스 모델을 다시 로드해야 PEFT 적용 가능
    base_model_for_peft = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16)
    finetuned_model = PeftModel.from_pretrained(base_model_for_peft, ADAPTER_PATH)
    finetuned_model = finetuned_model.merge_and_unload()
    
    ft_results_df = evaluate_model("Finetuned Model", finetuned_model, tokenizer, test_data)

    # 결과 저장 및 출력
    ft_avg_scores = ft_results_df[[col for col in ft_results_df.columns if "BERTScore" in col]].mean()
    print("\n--- [Finetuned Model] 전체 평균 점수 ---\
", ft_avg_scores)
    ft_results_df.to_markdown("./finetuned_model_evaluation02.md", index=False)
    print("✅ 파인튜닝 모델 평가 결과가 finetuned_model_evaluation02.md 파일에 저장되었습니다.")

if __name__ == "__main__":
    main()


✅ 총 40개의 평가 데이터를 로드했습니다.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- [Base Model] 모델 답변 생성 시작 ---


평가 중 [Base Model]: 100%|██████████| 40/40 [02:50<00:00,  4.26s/it]


--- [Base Model] BERTScore 계산 중 ---
✅ [Base Model] 평가 완료

--- [Base Model] 전체 평균 점수 --- BERTScore-P     0.648318
BERTScore-R     0.680015
BERTScore-F1    0.662373
dtype: float64
✅ 베이스 모델 평가 결과가 improved_base_model_evaluation02.md 파일에 저장되었습니다.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- [Finetuned Model] 모델 답변 생성 시작 ---


평가 중 [Finetuned Model]: 100%|██████████| 40/40 [02:49<00:00,  4.24s/it]


--- [Finetuned Model] BERTScore 계산 중 ---
✅ [Finetuned Model] 평가 완료

--- [Finetuned Model] 전체 평균 점수 --- BERTScore-P     0.676636
BERTScore-R     0.705634
BERTScore-F1    0.689715
dtype: float64
✅ 파인튜닝 모델 평가 결과가 finetuned_model_evaluation02.md 파일에 저장되었습니다.


In [3]:
# -*- coding: utf-8 -*-
"""
(수정-v3) 정량 평가 스크립트.

변경 사항:
- F1-Score 외에 Precision, Recall 점수를 모두 결과에 포함하여 다각적인 분석이 가능하도록 함.
- 최종 보고서에 세 가지 지표(P, R, F1)를 모두 나란히 비교하여 모델의 성향을 파악하기 용이하게 함.
"""

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json
import bert_score
import pandas as pd
from tqdm import tqdm

# --- 공통 설정 ---
BASE_MODEL_PATH = "./exaone_4.0_1.2b"
ADAPTER_PATH = "./llm_finetuned_model"
TEST_DATA_PATH = './test.jsonl'
OUTPUT_MD_PATH = "./quantitative_side_by_side_evaluation.md"

def evaluate_model(model_name, model, tokenizer, data):
    """주어진 모델에 대해 평가를 수행하고 결과 데이터프레임을 반환하는 함수"""
    print(f"\n--- [{model_name}] 모델 답변 생성 시작 ---")
    
    predictions, references, questions = [], [], []

    for item in tqdm(data, desc=f"평가 중 [{model_name}]"):
        question = item['messages'][0]['content']
        reference_answer = item['messages'][1]['content']
        
        messages = [{"role": "user", "content": question}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        
        output = model.generate(
            input_ids, max_new_tokens=256, do_sample=False, eos_token_id=tokenizer.eos_token_id
        )
        
        full_text = tokenizer.decode(output[0], skip_special_tokens=True)
        answer = full_text.split("assistant\n")[1].strip() if "assistant\n" in full_text else full_text

        predictions.append(answer)
        references.append(reference_answer)
        questions.append(question)

    print(f"--- [{model_name}] BERTScore 계산 중 ---")
    bert_p, bert_r, bert_f1 = bert_score.score(
        predictions, references, lang="ko", model_type="bert-base-multilingual-cased", verbose=False
    )
    
    df = pd.DataFrame({
        "Question": questions,
        f"{model_name}_Answer": predictions,
        f"{model_name}_Precision": bert_p.tolist(),
        f"{model_name}_Recall": bert_r.tolist(),
        f"{model_name}_F1_Score": bert_f1.tolist(),
        "Reference Answer": references
    })
    print(f"✅ [{model_name}] 평가 완료")
    return df

def main():
    """메인 실행 함수"""
    with open(TEST_DATA_PATH, 'r', encoding='utf-8') as f:
        test_data = [json.loads(line) for line in f if line.strip()]
    print(f"✅ 총 {len(test_data)}개의 평가 데이터를 로드했습니다.")

    # --- 모델 평가 ---
    base_model, tokenizer = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16), AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
    base_results_df = evaluate_model("Base", base_model, tokenizer, test_data)
    del base_model
    torch.cuda.empty_cache()

    base_model_for_peft = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, device_map="auto", torch_dtype=torch.bfloat16)
    finetuned_model = PeftModel.from_pretrained(base_model_for_peft, ADAPTER_PATH).merge_and_unload()
    ft_results_df = evaluate_model("Finetuned", finetuned_model, tokenizer, test_data)
    del base_model_for_peft, finetuned_model
    torch.cuda.empty_cache()

    # --- 결과 병합 및 저장 ---
    comparison_df = pd.merge(base_results_df, ft_results_df, on=["Question", "Reference Answer"])
    
    # 컬럼 순서 재정렬
    comparison_df = comparison_df[[
        "Question", "Reference Answer", 
        "Base_Answer", "Base_Precision", "Base_Recall", "Base_F1_Score",
        "Finetuned_Answer", "Finetuned_Precision", "Finetuned_Recall", "Finetuned_F1_Score"
    ]]

    # 전체 평균 점수 계산
    avg_scores_text = "| Metric | Base Model | Finetuned Model |\n"
    avg_scores_text += "|---|---|---|"
    for metric in ["Precision", "Recall", "F1_Score"]:
        base_avg = comparison_df[f'Base_{metric}'].mean()
        ft_avg = comparison_df[f'Finetuned_{metric}'].mean()
        avg_scores_text += f"| **{metric}** | {base_avg:.4f} | {ft_avg:.4f} |\n"

    print("\n--- 전체 평균 점수 ---")
    print(avg_scores_text)

    # Markdown 파일로 저장
    with open(OUTPUT_MD_PATH, 'w', encoding='utf-8') as f:
        f.write("# 정량 평가 비교 (P, R, F1) (v2)\n\n")
        f.write("## 전체 평균 점수\n\n")
        f.write(avg_scores_text)
        f.write("\n\n## 개별 결과 비교\n\n")
        f.write(comparison_df.to_markdown(index=False))

    print(f"\n✅ 비교 평가 결과가 '{OUTPUT_MD_PATH}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    main()

✅ 총 40개의 평가 데이터를 로드했습니다.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- [Base] 모델 답변 생성 시작 ---


평가 중 [Base]: 100%|██████████| 40/40 [02:50<00:00,  4.27s/it]


--- [Base] BERTScore 계산 중 ---


...

✅ [Base] 평가 완료


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- [Finetuned] 모델 답변 생성 시작 ---


평가 중 [Finetuned]: 100%|██████████| 40/40 [02:55<00:00,  4.39s/it]


--- [Finetuned] BERTScore 계산 중 ---
✅ [Finetuned] 평가 완료

--- 전체 평균 점수 ---
| Metric | Base Model | Finetuned Model |
|---|---|---|| **Precision** | 0.6707 | 0.6806 |
| **Recall** | 0.7284 | 0.7343 |
| **F1_Score** | 0.6977 | 0.7058 |


✅ 비교 평가 결과가 './quantitative_side_by_side_evaluation.md' 파일에 저장되었습니다.
